In [1]:
import numpy as np
import torch

In [2]:
train_data = torch.load('/project/annotation/train.pt')
test_data = torch.load('/project/annotation/test.pt')
valid_data = torch.load('/project/annotation/valid.pt')

In [3]:
# dataloaders
batch_size = 16

# make sure to SHUFFLE for your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

NameError: name 'DataLoader' is not defined